In [1]:
import asyncio
import json
import logging
import time
from dataclasses import dataclass
from enum import Enum
from pathlib import Path
from typing import Any, Dict, List, Optional

import aiohttp
import numpy as np
import pandas as pd
import pendulum
import requests
from sqlalchemy import (BigInteger, Boolean, Column, Float, Integer, String,
                        and_, create_engine, select, text)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base, sessionmaker
from utils.models import SlSpots
from utils.schemas import SlApiEndpoints, SlApiParams
from utils.sl_data import SurflineSpots
from utils.utils import LOCAL_AIRFLOW_PG_URI, LOCAL_PG_URI

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
Base = declarative_base()

In [5]:
engine = create_engine(LOCAL_AIRFLOW_PG_URI)
SessionLocal = sessionmaker(bind=engine)

## Note:
See `240220_sl_surf_spots.ipynb` for spot getter

In [6]:
response = requests.get("https://services.surfline.com/taxonomy?type=taxonomy&id=58f7ed51dadb30820bb3879c&maxDepth=0")

* You will not get Surfline forecast data without a valid Surfline premium login. Add your credentials to `.env.development`:
  ```
  SURFLINE_EMAIL=xxx
  SURFLINE_PASSWORD=yyy
  ```

##### Requests

`https://services.surfline.com/kbyg/spots/forecasts/{type}?{params}`


Type|Data
----|----
rating|array of human-readable and numeric (0-6) ratings
wave|array of min/max sizes & optimal scores
wind|array of wind directions/speeds & optimal scores
tides|array of types & heights
weather|array of sunrise/set times, array of temperatures/weather conditions

Param|Values|Effect
-----|------|------
spotId|string|Surfline spot id that you want data for. A typical Surfline URL is `https://www.surfline.com/surf-report/venice-breakwater/590927576a2e4300134fbed8` where `590927576a2e4300134fbed8` is the `spotId`
days|integer|Number of forecast days to get (Max 6 w/o access token, Max 17 w/ premium token)
intervalHours|integer|Minimum of 1 (hour)
maxHeights|boolean|`true` seems to remove min & optimal values from the wave data output
sds|boolean|If true, use the new LOTUS forecast engine
accesstoken|string|Auth token to get premium data access (optional)

Anywhere there is an `optimalScore` the value can be interpreted as follows:

Value|Meaning
-----|-------
0|Suboptimal
1|Good
2|Optimal


In [7]:
types = ["rating", "wave", "wind", "tides", "weather"]
params = ["spotId", "days", "intervalHours", "maxHeights", "sds", "accesstoken"]
base = "https://services.surfline.com/kbyg/spots/forecasts"

In [8]:
datapath = Path('./data')


In [9]:
df = pd.read_csv(datapath/'spot_list.csv')

In [10]:
df.head()

,Unnamed: 0,ids,names,lon,lat,urls
0,0,584204204e65fad6a7709b5d,Dauphin Island,-88.11700,30.229000,https://www.surfline.com/surf-report/dauphin-i...
1,1,584204204e65fad6a7709b61,Spuds,-87.54900,30.273000,https://www.surfline.com/surf-report/spuds/584...
2,2,584204204e65fad6a7709b62,Alabama Point,-87.56200,30.270000,https://www.surfline.com/surf-report/alabama-p...
3,3,584204204e65fad6a7709b60,West Pass,-87.73700,30.239000,https://www.surfline.com/surf-report/west-pass...
4,4,65948156c329a78a0914a15e,Morgantown Beach,-87.91913,30.230299,https://www.surfline.com/surf-report/morgantow...


Get the spot `id` for 1st Street Jetty in Va Beach

In [11]:
jetty_id = df[df['names'].str.contains('1st Street Jetty', case=False, na=False)]['ids'].values[0]
jetty_id

'584204214e65fad6a7709ce7'

In [12]:
ex_params = {params[0]: jetty_id}
ex_params

{'spotId': '584204214e65fad6a7709ce7'}

Surfline seems to change their spot IDs periodically. Check a spot on the website and pass the objectId from the url as a param to debug if this is the case. If they've changed you'll need to run the notebook `240220_sl_surf_spots.ipynb` as mentioned above to refresh the spots dataset

In [13]:
debug_params = {params[0]: "584204214e65fad6a7709ce7"}

In [14]:
res = requests.get(f"{base}/{types[0]}", params=ex_params)
res.status_code

200

In [15]:
rating_json = res.json()

In [16]:
four_day_json = res.json()
if 'data' in four_day_json and 'rating' in four_day_json['data']:
    four_day_json['data']['rating'] = four_day_json['data']['rating'][:24]

In [17]:
def cull_extra_days(full_json):
    if 'data' in full_json and 'rating' in full_json['data']:
        full_json['data']['rating'] = full_json['data']['rating'][:24]

Drop extra days of forecast

In [18]:
cull_extra_days(four_day_json)

In [19]:
len(four_day_json['data']['rating'])

24

Convert a unix timestamp -> utc

In [20]:
pendulum.from_timestamp(rating_json['data']['rating'][0]['timestamp'], 'UTC')

DateTime(2024, 6, 10, 4, 0, 0, tzinfo=Timezone('UTC'))

In [21]:
pendulum.from_timestamp(rating_json['data']['rating'][int(72 / 3)-1]['timestamp'], 'UTC')

DateTime(2024, 6, 11, 3, 0, 0, tzinfo=Timezone('UTC'))

The `utcOffset` field seems to be aware that I'm working in EST currently. Either that or it's the time coding for the spot itself.

Let's check a west coast spot to confirm how this is handled

In [22]:
df

,Unnamed: 0,ids,names,lon,lat,urls
0,0,584204204e65fad6a7709b5d,Dauphin Island,-88.117000,30.229000,https://www.surfline.com/surf-report/dauphin-i...
1,1,584204204e65fad6a7709b61,Spuds,-87.549000,30.273000,https://www.surfline.com/surf-report/spuds/584...
2,2,584204204e65fad6a7709b62,Alabama Point,-87.562000,30.270000,https://www.surfline.com/surf-report/alabama-p...
3,3,584204204e65fad6a7709b60,West Pass,-87.737000,30.239000,https://www.surfline.com/surf-report/west-pass...
4,4,65948156c329a78a0914a15e,Morgantown Beach,-87.919130,30.230299,https://www.surfline.com/surf-report/morgantow...
...,...,...,...,...,...,...
1298,1298,640a2d14451905376297f483,Rutherford Beach,-93.124300,29.758500,https://www.surfline.com/surf-report/rutherfor...
1299,1299,5842041f4e65fad6a7708a1a,Assateague,-75.177040,38.148058,https://www.surfline.com/surf-report/assateagu...
1300,1300,5842041f4e65fad6a770886d,Ocean City Boardwalk,-75.081170,38.338461,https://www.surfline.com/surf-report/ocean-cit...
1301,1301,5842041f4e65fad6a7708a1b,North End to Ocean City Inlet,-75.080177,38.338890,https://www.surfline.com/surf-report/north-end...


In [23]:
la_jolla_id = df[df['names'].str.contains("La Jolla", case=False, na=False)]['ids'].values[0]
la_jolla_dict = {params[0]: la_jolla_id}

In [24]:
la_jolla_dict

{'spotId': '5842041f4e65fad6a77088cc'}

In [25]:
pendulum.now("utc")

DateTime(2024, 6, 10, 12, 5, 26, 445472, tzinfo=Timezone('UTC'))

In [26]:
new_dict = {"spot_id": "test", "spot_name": "test_2", "date": pendulum.now("utc"), "forecast": four_day_json}

In [27]:
def fetch_from_sl_api(endpoint: SlApiEndpoints, param_type: SlApiParams, param: str):
    base_url = "https://services.surfline.com/kbyg/spots/forecasts"
    res = requests.get(f"{base_url}/{endpoint}", params={param_type: param})
    data = res.json()
    return data

In [28]:
test_res = fetch_from_sl_api(SlApiEndpoints.RATING.value, SlApiParams.SPOT_ID.value, param=jetty_id)

In [29]:
spot_ratings = []
for spot_id, spot_name in df[['ids', 'names']][:3].values:
    res = requests.get(f"{base}/rating", params={'spotId': spot_id})
    data = res.json()
    cull_extra_days(data)
    current_date = pendulum.now("utc")
    utc_date = current_date.strftime("%Y-%m-%d")
    data['spot_id'] = spot_id
    data['spot_name'] = spot_name
    data['utc_fetch_date'] = utc_date
    spot_ratings.append(data)
    # time.sleep()

In [30]:
pendulum.from_timestamp(rating_json['data']['rating'][0]['timestamp'], 'UTC')

DateTime(2024, 6, 10, 4, 0, 0, tzinfo=Timezone('UTC'))

In [31]:
ratings_df = pd.json_normalize(spot_ratings, record_path=['data', 'rating'], meta=['spot_id', 'spot_name', 'utc_fetch_date'] )

In [32]:
ratings_df

,timestamp,utcOffset,rating.key,rating.value,spot_id,spot_name,utc_fetch_date
0,1717995600,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
1,1717999200,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
2,1718002800,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
3,1718006400,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
4,1718010000,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
...,...,...,...,...,...,...,...
67,1718064000,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
68,1718067600,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
69,1718071200,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
70,1718074800,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10


In [33]:
ratings_df['timestamp'] = ratings_df['timestamp'].apply(lambda x: pendulum.from_timestamp(x).to_datetime_string())

Alright, so it looks like each spot's forecast starts at 12am *local time*, with the timestamp for that time in unix. To figure out the flat `UTC` time for each spot you can just apply the `utcOffset` that is included in response. 

In [34]:
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'])

In [35]:
ratings_df

,timestamp,utcOffset,rating.key,rating.value,spot_id,spot_name,utc_fetch_date
0,2024-06-10 05:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
1,2024-06-10 06:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
2,2024-06-10 07:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
3,2024-06-10 08:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
4,2024-06-10 09:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10
...,...,...,...,...,...,...,...
67,2024-06-11 00:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
68,2024-06-11 01:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
69,2024-06-11 02:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10
70,2024-06-11 03:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10


In [36]:
ratings_df['timestamp_utc'] = ratings_df.apply(lambda row: row['timestamp'] + pd.Timedelta(hours=row['utcOffset']), axis=1)

In [37]:
ratings_df

,timestamp,utcOffset,rating.key,rating.value,spot_id,spot_name,utc_fetch_date,timestamp_utc
0,2024-06-10 05:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 00:00:00
1,2024-06-10 06:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 01:00:00
2,2024-06-10 07:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 02:00:00
3,2024-06-10 08:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 03:00:00
4,2024-06-10 09:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b5d,Dauphin Island,2024-06-10,2024-06-10 04:00:00
...,...,...,...,...,...,...,...,...
67,2024-06-11 00:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 19:00:00
68,2024-06-11 01:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 20:00:00
69,2024-06-11 02:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 21:00:00
70,2024-06-11 03:00:00,-5,VERY_POOR,0,584204204e65fad6a7709b62,Alabama Point,2024-06-10,2024-06-10 22:00:00


In [38]:
ratings_df.dtypes

timestamp         datetime64[ns]
utcOffset                  int64
rating.key                object
rating.value               int64
spot_id                   object
spot_name                 object
utc_fetch_date            object
timestamp_utc     datetime64[ns]
dtype: object

In [39]:
with SessionLocal() as db:
    matching_spots = []
    for spot in df['names']:
        stmt = text("""select * from spots where spot_name like :spot""")
        result = db.execute(stmt, {"spot": spot}).fetchall()
        if len(result) > 0:
            matching_spots.append(result)
    

In [40]:
len(matching_spots)

1302

In [41]:
jetty_waves = fetch_from_sl_api(SlApiEndpoints.WAVE.value, SlApiParams.SPOT_ID.value, jetty_id)

In [42]:
jetty_waves['associated']

{'units': {'temperature': 'F',
  'tideHeight': 'FT',
  'swellHeight': 'FT',
  'waveHeight': 'FT',
  'windSpeed': 'KTS',
  'pressure': 'MB'},
 'utcOffset': -4,
 'location': {'lon': -75.96648, 'lat': 36.83036135089083},
 'forecastLocation': {'lon': -75.947, 'lat': 36.847},
 'offshoreLocation': {'lon': -75.75, 'lat': 37},
 'runInitializationTimestamp': 1717977600}

In [43]:
jetty_waves['associated']['spotId'] = jetty_id

In [44]:
jetty_waves['data']['spotId'] = jetty_id

In [80]:
jetty_meta_df = pd.json_normalize(jetty_waves)

In [81]:
jetty_meta_df.drop(['permissions.violations', 'permissions.data', 'data.wave', 'data.spotId'], inplace=True, axis=1)

In [46]:
jetty_meta_df

,associated.units.temperature,associated.units.tideHeight,associated.units.swellHeight,associated.units.waveHeight,associated.units.windSpeed,associated.units.pressure,associated.utcOffset,associated.location.lon,associated.location.lat,associated.forecastLocation.lon,associated.forecastLocation.lat,associated.offshoreLocation.lon,associated.offshoreLocation.lat,associated.runInitializationTimestamp,associated.spotId
0,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,36.847,-75.75,37,1717977600,584204214e65fad6a7709ce7


In [47]:
jetty_wave_df = pd.json_normalize(
    jetty_waves, record_path=["data", "wave"], meta=[["data", "spotId"]]
)
jetty_wave_df.drop("swells", inplace=True, axis=1)
jetty_wave_df.rename(columns={"power": "wave_power"}, inplace=True)

In [48]:
jetty_swell_df = pd.json_normalize(
    jetty_waves,
    record_path=['data', 'wave', 'swells'],
    meta=[['data', 'wave', 'timestamp'], ['data', 'spotId']]
)

jetty_swell_df['swells_idx'] = jetty_swell_df.groupby('data.wave.timestamp').cumcount()

In [49]:
jetty_swell_df.head()

,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,data.spotId,swells_idx
0,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,0
1,1.34514,6,0.6454,7.33609,138.02,128.410,0,1717992000,584204214e65fad6a7709ce7,1
2,0.45932,9,0.3126,0.96313,74.20,69.005,0,1717992000,584204214e65fad6a7709ce7,2
3,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,3
4,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,4


In [50]:
jetty_swell_df.rename({"power": 'swell_power'}, inplace=True)

In [51]:
jetty_swell_df['data.wave.timestamp'].value_counts()

1717992000    6
1717995600    6
1718308800    6
1718305200    6
1718301600    6
             ..
1718118000    6
1718114400    6
1718110800    6
1718107200    6
1718420400    6
Name: data.wave.timestamp, Length: 120, dtype: int64

In [52]:
jetty_meta_df.head()

,associated.units.temperature,associated.units.tideHeight,associated.units.swellHeight,associated.units.waveHeight,associated.units.windSpeed,associated.units.pressure,associated.utcOffset,associated.location.lon,associated.location.lat,associated.forecastLocation.lon,associated.forecastLocation.lat,associated.offshoreLocation.lon,associated.offshoreLocation.lat,associated.runInitializationTimestamp,associated.spotId
0,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,36.847,-75.75,37,1717977600,584204214e65fad6a7709ce7


In [53]:
jetty_wave_df.head()

,timestamp,probability,utcOffset,wave_power,surf.min,surf.max,surf.plus,surf.humanRelation,surf.raw.min,surf.raw.max,surf.optimalScore,data.spotId
0,1717992000,100.000000,-4,8.29922,1,2,False,Knee to thigh,0.03281,1.03281,0,584204214e65fad6a7709ce7
1,1717995600,100.000000,-4,5.13194,1,2,False,Knee to thigh,0.03281,1.03281,0,584204214e65fad6a7709ce7
2,1717999200,100.000000,-4,8.76772,1,2,False,Knee to thigh,0.06562,1.03281,0,584204214e65fad6a7709ce7
3,1718002800,96.666667,-4,5.51770,1,2,False,Knee to thigh,0.06562,1.03281,0,584204214e65fad6a7709ce7
4,1718006400,96.666667,-4,5.29797,1,2,False,Knee to thigh,0.03281,1.03281,0,584204214e65fad6a7709ce7


In [54]:
jetty_swell_df.head(n=10)

,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,data.spotId,swells_idx
0,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,0
1,1.34514,6,0.6454,7.33609,138.02,128.410,0,1717992000,584204214e65fad6a7709ce7,1
2,0.45932,9,0.3126,0.96313,74.20,69.005,0,1717992000,584204214e65fad6a7709ce7,2
3,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,3
4,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,584204214e65fad6a7709ce7,4
5,0.68898,3,0.0000,0.00000,4.98,-12.885,0,1717992000,584204214e65fad6a7709ce7,5
6,0.62336,2,0.0351,0.01456,19.55,9.195,0,1717995600,584204214e65fad6a7709ce7,0
7,0.19685,3,0.0000,0.00000,281.95,274.545,0,1717995600,584204214e65fad6a7709ce7,1
8,0.45932,9,0.2286,0.70432,75.04,69.505,0,1717995600,584204214e65fad6a7709ce7,2
9,1.04987,5,0.3495,2.04041,151.70,145.145,0,1717995600,584204214e65fad6a7709ce7,3


In [55]:
combined_waves_df = pd.merge(
    jetty_wave_df,
    jetty_swell_df,
    how="inner",
    left_on=["timestamp", "data.spotId"],
    right_on=["data.wave.timestamp", 'data.spotId'],
)

In [56]:
len(combined_waves_df)

720

In [57]:
combined_waves_df.head()

,timestamp,probability,utcOffset,wave_power,surf.min,surf.max,surf.plus,surf.humanRelation,surf.raw.min,surf.raw.max,...,data.spotId,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,swells_idx
0,1717992000,100.0,-4,8.29922,1,2,False,Knee to thigh,0.03281,1.03281,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,0
1,1717992000,100.0,-4,8.29922,1,2,False,Knee to thigh,0.03281,1.03281,...,584204214e65fad6a7709ce7,1.34514,6,0.6454,7.33609,138.02,128.410,0,1717992000,1
2,1717992000,100.0,-4,8.29922,1,2,False,Knee to thigh,0.03281,1.03281,...,584204214e65fad6a7709ce7,0.45932,9,0.3126,0.96313,74.20,69.005,0,1717992000,2
3,1717992000,100.0,-4,8.29922,1,2,False,Knee to thigh,0.03281,1.03281,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,3
4,1717992000,100.0,-4,8.29922,1,2,False,Knee to thigh,0.03281,1.03281,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,4


In [58]:
combined_df = pd.merge(jetty_meta_df, combined_waves_df, how='cross')

In [59]:
combined_df

,associated.units.temperature,associated.units.tideHeight,associated.units.swellHeight,associated.units.waveHeight,associated.units.windSpeed,associated.units.pressure,associated.utcOffset,associated.location.lon,associated.location.lat,associated.forecastLocation.lon,...,data.spotId,height,period,impact,power,direction,directionMin,optimalScore,data.wave.timestamp,swells_idx
0,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,0
1,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,1.34514,6,0.6454,7.33609,138.02,128.410,0,1717992000,1
2,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.45932,9,0.3126,0.96313,74.20,69.005,0,1717992000,2
3,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,3
4,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1718420400,1
716,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.42651,16,0.2291,1.87524,102.81,99.040,0,1718420400,2
717,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,3.60892,5,0.5685,32.41157,160.76,145.185,0,1718420400,3
718,F,FT,FT,FT,KTS,MB,-4,-75.96648,36.830361,-75.947,...,584204214e65fad6a7709ce7,0.42651,10,0.1813,0.61458,105.04,99.895,0,1718420400,4


In [60]:
engine = create_engine(LOCAL_PG_URI)
SessionLocal = sessionmaker(bind=engine)

In [61]:
with SessionLocal() as db:
    stmt = select(SlSpots.spot_id)
    spots = db.execute(stmt).scalars().all()

In [62]:
def transform_sl_wave_data(data: Dict) -> pd.DataFrame:
    if not data:
        raise ValueError("Data is empty")

    meta_df = pd.json_normalize(data)
    meta_df.drop(
        ["permissions.violations", "permissions.data", "data.wave", "data.spotId"],
        inplace=True,
        axis=1,
    )

    wave_df = pd.json_normalize(
        jetty_waves, record_path=["data", "wave"], meta=[["data", "spotId"]]
    )
    wave_df.drop("swells", inplace=True, axis=1)
    wave_df.rename(columns={"power": "wave_power"}, inplace=True)

    swell_df = pd.json_normalize(
        jetty_waves,
        record_path=["data", "wave", "swells"],
        meta=[["data", "wave", "timestamp"], ["data", "spotId"]],
    )

    swell_df.rename(columns={"power": "swell_power"}, inplace=True)
    swell_df["swells_idx"] = swell_df.groupby("data.wave.timestamp").cumcount()

    combined_waves_df = pd.merge(
        wave_df,
        swell_df,
        how="inner",
        left_on=["timestamp", "data.spotId"],
        right_on=["data.wave.timestamp", "data.spotId"],
    )

    combined_df = pd.merge(meta_df, combined_waves_df, how='cross')

    return combined_df

In [63]:
data = []
for spot in spots[:2]:
    result = fetch_from_sl_api(SlApiEndpoints.WAVE.value, SlApiParams.SPOT_ID.value, param=spot)
    if result.get("associated"):
        result['associated']['spotId'] = spot
        result['data']['spotId'] = spot
    data.append(result)


In [64]:
full_df = pd.concat([transform_sl_wave_data(entry) for entry in data])

In [65]:
full_df

,associated.units.temperature,associated.units.tideHeight,associated.units.swellHeight,associated.units.waveHeight,associated.units.windSpeed,associated.units.pressure,associated.utcOffset,associated.location.lon,associated.location.lat,associated.forecastLocation.lon,...,data.spotId,height,period,impact,swell_power,direction,directionMin,optimalScore,data.wave.timestamp,swells_idx
0,F,FT,FT,FT,KTS,MB,-7,-124.6654,48.3105,-124.6654,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,0
1,F,FT,FT,FT,KTS,MB,-7,-124.6654,48.3105,-124.6654,...,584204214e65fad6a7709ce7,1.34514,6,0.6454,7.33609,138.02,128.410,0,1717992000,1
2,F,FT,FT,FT,KTS,MB,-7,-124.6654,48.3105,-124.6654,...,584204214e65fad6a7709ce7,0.45932,9,0.3126,0.96313,74.20,69.005,0,1717992000,2
3,F,FT,FT,FT,KTS,MB,-7,-124.6654,48.3105,-124.6654,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,3
4,F,FT,FT,FT,KTS,MB,-7,-124.6654,48.3105,-124.6654,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1717992000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,F,FT,FT,FT,KTS,MB,-7,-124.6830,48.2717,-124.6830,...,584204214e65fad6a7709ce7,0.00000,0,0.0000,0.00000,0.00,0.000,0,1718420400,1
716,F,FT,FT,FT,KTS,MB,-7,-124.6830,48.2717,-124.6830,...,584204214e65fad6a7709ce7,0.42651,16,0.2291,1.87524,102.81,99.040,0,1718420400,2
717,F,FT,FT,FT,KTS,MB,-7,-124.6830,48.2717,-124.6830,...,584204214e65fad6a7709ce7,3.60892,5,0.5685,32.41157,160.76,145.185,0,1718420400,3
718,F,FT,FT,FT,KTS,MB,-7,-124.6830,48.2717,-124.6830,...,584204214e65fad6a7709ce7,0.42651,10,0.1813,0.61458,105.04,99.895,0,1718420400,4


In [66]:
full_df.columns

Index(['associated.units.temperature', 'associated.units.tideHeight',
       'associated.units.swellHeight', 'associated.units.waveHeight',
       'associated.units.windSpeed', 'associated.units.pressure',
       'associated.utcOffset', 'associated.location.lon',
       'associated.location.lat', 'associated.forecastLocation.lon',
       'associated.forecastLocation.lat', 'associated.offshoreLocation.lon',
       'associated.offshoreLocation.lat',
       'associated.runInitializationTimestamp', 'associated.spotId',
       'timestamp', 'probability', 'utcOffset', 'wave_power', 'surf.min',
       'surf.max', 'surf.plus', 'surf.humanRelation', 'surf.raw.min',
       'surf.raw.max', 'surf.optimalScore', 'data.spotId', 'height', 'period',
       'impact', 'swell_power', 'direction', 'directionMin', 'optimalScore',
       'data.wave.timestamp', 'swells_idx'],
      dtype='object')

In [67]:
full_df.dtypes

associated.units.temperature              object
associated.units.tideHeight               object
associated.units.swellHeight              object
associated.units.waveHeight               object
associated.units.windSpeed                object
associated.units.pressure                 object
associated.utcOffset                       int64
associated.location.lon                  float64
associated.location.lat                  float64
associated.forecastLocation.lon          float64
associated.forecastLocation.lat          float64
associated.offshoreLocation.lon          float64
associated.offshoreLocation.lat          float64
associated.runInitializationTimestamp      int64
associated.spotId                         object
timestamp                                 object
probability                              float64
utcOffset                                  int64
wave_power                               float64
surf.min                                   int64
surf.max            

In [68]:
class SlRatings(Base):
    __tablename__ = 'sl_ratings'

    id = Column(Integer, primary_key=True, autoincrement=True)
    associated_units_temperature = Column(String)
    associated_units_tideHeight = Column(String)
    associated_units_swellHeight = Column(String)
    associated_units_waveHeight = Column(String)
    associated_units_windSpeed = Column(String)
    associated_units_pressure = Column(String)
    associated_utcOffset = Column(Integer)
    associated_location_lon = Column(Float)
    associated_location_lat = Column(Float)
    associated_forecastLocation_lon = Column(Float)
    associated_forecastLocation_lat = Column(Float)
    associated_offshoreLocation_lon = Column(Float)
    associated_offshoreLocation_lat = Column(Float)
    associated_runInitializationTimestamp = Column(BigInteger)
    associated_spotId = Column(String)
    timestamp = Column(String)
    probability = Column(Float)
    utcOffset = Column(Integer)
    wave_power = Column(Float)
    surf_min = Column(Integer)
    surf_max = Column(Integer)
    surf_plus = Column(Boolean)
    surf_humanRelation = Column(String)
    surf_raw_min = Column(Float)
    surf_raw_max = Column(Float)
    surf_optimalScore = Column(Integer)
    data_spotId = Column(String)
    height = Column(Float)
    period = Column(Integer)
    impact = Column(Float)
    swell_power = Column(Float)
    direction = Column(Float)
    directionMin = Column(Float)
    optimalScore = Column(Integer)
    data_wave_timestamp = Column(String)
    swells_idx = Column(Integer)


In [69]:
def create_tables():
    Base.metadata.create_all(bind=engine)

In [70]:
create_tables()

In [82]:
class SpotForecast:
    def __init__(self, database_uri):
        self.spots = []
        self.engine = create_engine(database_uri)
        self.SessionLocal = sessionmaker(bind=engine)

    def get_session(self):
        return self.SessionLocal()

    def fetch_all_forecasts(self) -> List[Dict[Any, Any]]:
        data = []
        for spot in self.spots[:2]:
            result = self.fetch_forecast(
                SlApiEndpoints.WAVE.value, SlApiParams.SPOT_ID.value, param=spot
            )
            if result.get("associated"):
                result["associated"]["spotId"] = spot
                result["data"]["spotId"] = spot
            data.append(result)
        return data

    def fetch_forecast(self, endpoint: SlApiEndpoints, param_type: SlApiParams, param: str):
        base_url = "https://services.surfline.com/kbyg/spots/forecasts"
        res = requests.get(f"{base_url}/{endpoint}", params={param_type: param})
        data = res.json()
        return data

    def fetch_spots_from_db(self):
        with self.get_session() as db:
            stmt = select(SlSpots.spot_id)
            self.spots = db.execute(stmt).scalars().all()

    def transform_wave_data(self, data: Dict) -> pd.DataFrame:
        if not data:
            raise ValueError("Data is empty")

        meta_df = pd.json_normalize(data, sep="_")
        meta_df.drop(
            ["permissions_violations", "permissions_data", "data_wave", "data_spotId"],
            inplace=True,
            axis=1,
        )

        wave_df = pd.json_normalize(
            jetty_waves, record_path=["data", "wave"], meta=[["data", "spotId"]], sep="_"
        )
        wave_df.drop("swells", inplace=True, axis=1)
        wave_df.rename(columns={"power": "wave_power"}, inplace=True)

        swell_df = pd.json_normalize(
            jetty_waves,
            record_path=["data", "wave", "swells"],
            meta=[["data", "wave", "timestamp"], ["data", "spotId"]],
            sep="_",
        )

        swell_df.rename(columns={"power": "swell_power"}, inplace=True)
        swell_df["swells_idx"] = swell_df.groupby("data_wave_timestamp").cumcount()

        combined_waves_df = pd.merge(
            wave_df,
            swell_df,
            how="inner",
            left_on=["timestamp", "data_spotId"],
            right_on=["data_wave_timestamp", "data_spotId"],
        )

        combined_df = pd.merge(meta_df, combined_waves_df, how="cross")

        return combined_df

    def transform_all_wave_data(self):
        # data = self.fetch_all_forecasts()
        # dfs = [self.transform_wave_data(entry) for entry in data]
        pass

    def load_to_pg(self):
        with self.get_session() as db:
            pass

In [83]:
spot_forecast = SpotForecast(LOCAL_PG_URI)

In [87]:
test_insert = spot_forecast.transform_wave_data(jetty_waves)

In [88]:
test_insert_dict = test_insert.to_dict('records')

In [90]:
with SessionLocal() as db:
    db.bulk_insert_mappings(SlRatings, test_insert_dict)
    db.commit()